# Create a slab with specified termination

Create a slab with a specified termination from a bulk material. 

<h2 style="color:green">Usage</h2>


1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. if prompted, select the termination for the slab in cell 2.5.
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

MILLER_INDICES = (0, 0, 1)
THICKNESS = 3  # in atomic layers
VACUUM = 10.0  # in angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
USE_ORTHOGONAL_C = True
USE_CONVENTIONAL_CELL = True

# Stoichiometric formula of the slab termination to be used.
SLAB_TERMINATION_FORMULA = None
# if None, the index of all possible terminations will be used
TERMINATION_INDEX = 0

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(material, repetitions=[3, 3, 1], rotation="0x")
visualize(material, repetitions=[3, 3, 1], rotation="-90x")

## 2. Configure slab and select termination

### 2.1. Create crystal lattice planes analyzer
CrystalLatticePlanesMaterialAnalyzer is used to analyze the crystal lattice planes of the material and determine possible terminations for the slab.


In [ ]:
from mat3ra.made.tools.analyze.lattice_planes import CrystalLatticePlanesMaterialAnalyzer

analyzer = CrystalLatticePlanesMaterialAnalyzer(material=material, miller_indices=MILLER_INDICES)

### 2.2. Get possible terminations for the slabs

In [ ]:
slab_terminations = analyzer.terminations

### 2.3. Visualize slabs for all possible terminations

In [ ]:

slabs = [analyzer.get_material_with_termination_without_vacuum(termination) for termination in slab_terminations]

visualize([{"material":slab, "title": slab.name} for slab in slabs ], repetitions=[3, 3, 1], rotation="-90x")

### 2.4. Print terminations for the Slab

In [ ]:
print("Terminations")
for idx, termination in enumerate(slab_terminations):
    print(f"    {idx}: {termination}")

### 2.5. Select termination for the Slab

In [ ]:
from mat3ra.made.tools.helpers import select_slab_termination
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

termination_index = TERMINATION_INDEX
termination = slab_terminations[termination_index]

if SLAB_TERMINATION_FORMULA:
    termination = select_slab_termination(slab_terminations, SLAB_TERMINATION_FORMULA)

if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    if sys.platform == "emscripten":
        termination = await ui_prompt_select_array_element_by_index_pyodide(slab_terminations, element_name="termination")
    else:
        termination = ui_prompt_select_array_element_by_index(slab_terminations, element_name="termination")

## 3. Create the slab with selected termination

3.1. Create slab with specified termination

In [ ]:
from mat3ra.made.tools.helpers import create_slab

slab = create_slab(crystal=material,
                   termination_top=termination,
                   use_conventional_cell=USE_CONVENTIONAL_CELL,
                   miller_indices=MILLER_INDICES,
                   number_of_layers= THICKNESS,
                   vacuum=VACUUM,
                   xy_supercell_matrix=XY_SUPERCELL_MATRIX,
                   use_orthogonal_c=USE_ORTHOGONAL_C,
                   )


## 4. Visualize the resulting slab

In [ ]:
visualize(slab, repetitions=[1, 1, 1], rotation="0x")

# 5. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(slab)